In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import seaborn as sns
#from itertools import izip

In [ ]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [ ]:
#train_data.groupby('hospital_death').hist()
#list_plot = list(np.arange(169,173))

In [ ]:
# df = train_data.iloc[:,list_plot]
# df['hospital_death'] = train_data['hospital_death']

In [ ]:
#df.groupby('hospital_death').hist()
#train_data[['height','hospital_death']].groupby('hospital_death').hist()

In [ ]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [ ]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [ ]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

#to_label_encode = list(data_dict[data_dict['Data Type'].isin(['string'])]['Variable Name'])

In [ ]:
cat_var = cat_var + ['weightclass']

In [ ]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [ ]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [ ]:
x_test = test_data.drop('hospital_death',axis=1)

In [ ]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
x_comb.head()

In [ ]:
list_drop = ['hospital_id','icu_id','encounter_id', 'patient_id', 'readmission_status']

In [ ]:
# # Remove Features with more than 70 percent missing values
#x_comb = x_comb.drop(['readmission_status'],axis = 1)
x_comb = x_comb.drop(list_drop,axis = 1)

In [ ]:
x_comb['hospital_admit_source'] = x_comb['hospital_admit_source'].replace({'Other ICU': 'ICU',
                                                                           'ICU to SDU':'SDU', 
                                                                           'Step-Down Unit (SDU)': 'SDU',        
                                                                           'Other Hospital':'Other',
                                                                           'Observation': 'Recovery Room',
                                                                           'Acute Care/Floor': 'Acute Care'})

In [ ]:
x_comb['icu_type'] = x_comb['icu_type'].replace({'CCU-CTICU': 
                                                 'Grpd_CICU', 
                                                 'CTICU':'Grpd_CICU', 
                                                 'Cardiac ICU':'Grpd_CICU'})


In [ ]:
x_comb['apache_2_bodysystem'] = x_comb['apache_2_bodysystem'].replace({'Undefined diagnoses': 
                                                                       'Undefined Diagnoses'})


In [ ]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] =  x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [ ]:
x_comb['new_bmi'] = (x_comb['weight']*10000)/(x_comb['height']*x_comb['height'])
x_comb[['bmi', 'new_bmi', 'weight', 'height']].head(10)

In [ ]:
x_comb['bmi'] = x_comb['bmi'].fillna(x_comb['new_bmi'])
x_comb[['bmi', 'new_bmi', 'weight', 'height']].head(10)

In [ ]:
x_comb = x_comb.drop(['new_bmi'], axis = 1)


In [ ]:
def get_sodium_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 111:
        return 4
    
    elif x>=111 and x<120:
        return 3
    
    elif x>=120 and x<130 :
        return 2
    
    elif x>=130 and x<150 :
        return 0
    
    elif x>=150 and x<155 :
        return 1
    
    elif x>=155 and x<160 :
        return 2
    
    elif x>=160 and x<180 :
        return 3
    
    elif x>=180 :
        return 4
        

In [ ]:
x_comb['sodium_apache'] = x_comb['sodium_apache'].apply(lambda x:get_sodium_apache(x))

In [ ]:
def get_creatinine_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 0.62:
        return 4
    
    elif x>=0.62 and x<1.47:
        return 0
    
    elif x>=1.47 and x<1.98 :
        return 4
    
    elif x>=1.98 and x<3.39 :
        return 2
    
    elif x>=3.39:
        return 8
    

In [ ]:
x_comb['creatinine_apache']  = x_comb['creatinine_apache'].apply(lambda x:get_creatinine_apache(x))

In [ ]:
def get_hematocrit_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 20:
        return 4
    
    elif x>=20 and x<30:
        return 2
    
    elif x>=30 and x<46 :
        return 0
    
    elif x>=46 and x<50 :
        return 1
    
    elif x>=50 and x<60 :
        return 2
    
    elif x>=60 :
        return 4
    

In [ ]:
x_comb['hematocrit_apache']  = x_comb['hematocrit_apache'].apply(lambda x:get_hematocrit_apache(x))

In [ ]:
x_comb['comb_gcs'] = x_comb['gcs_eyes_apache'] + x_comb['gcs_motor_apache'] + x_comb['gcs_verbal_apache']

In [ ]:
def get_map_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 50:
        return 4
    
    elif x>=50 and x<70:
        return 2
    
    elif x>=70 and x<110 :
        return 0
    
    elif x>=110 and x<130 :
        return 2
    
    elif x>=130 and x<160 :
        return 3
    
    elif x>=160 :
        return 4
        

In [ ]:
x_comb['map_apache'] = x_comb['map_apache'].apply(lambda x:get_map_apache(x))

In [ ]:
def get_heart_rate_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 40:
        return 4
    
    elif x>=40 and x<55:
        return 3
    
    elif x>=55 and x<110 :
        return 0
    
    elif x>=110 and x<140 :
        return 2
    
    elif x>=140 and x<180 :
        return 3
    
    elif x>=180 :
        return 4
        

In [ ]:
x_comb['heart_rate_apache'] = x_comb['heart_rate_apache'].apply(lambda x:get_heart_rate_apache(x))

In [ ]:
def get_resprate_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 6:
        return 4
    
    elif x>=6 and x<10:
        return 3
    
    elif x>=10 and x<12 :
        return 2
    
    elif x>=12 and x<25 :
        return 0
    
    elif x>=25 and x<35 :
        return 2
    
    elif x>=35 and x<50 :
        return 3
    
    elif x>=50 :
        return 4
        

In [ ]:
x_comb['resprate_apache'] = x_comb['resprate_apache'].apply(lambda x:get_resprate_apache(x))

In [ ]:
def get_pao2_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 200:
        return 0
    
    elif x>=200 and x<350:
        return 2
    
    elif x>=350 and x<500 :
        return 3
    
    elif x>=500 :
        return 4

In [ ]:
x_comb['pao2_apache'] = x_comb['pao2_apache'].apply(lambda x:get_pao2_apache(x))

In [ ]:
def get_ph_apache(x) : 
    
    if pd.isna(x):
        return np.nan
    
    if x < 7.15:
        return 4
    
    elif x>=7.15 and x<7.25:
        return 3
    
    elif x>=7.25 and x<7.33 :
        return 2
    
    elif x>=7.33 and x<7.5 :
        return 0
    
    elif x>=7.5 and x<7.6 :
        return 1
    
    elif x>=7.6 and x<7.7 :
        return 3
    
    elif x>=7.7 :
        return 4
        

In [ ]:
x_comb['ph_apache'] = x_comb['ph_apache'].apply(lambda x:get_ph_apache(x))

In [ ]:
x_comb['dis_ind'] = x_comb['aids'] + x_comb['cirrhosis'] + x_comb['diabetes_mellitus'] + x_comb['hepatic_failure'] + x_comb['immunosuppression'] + x_comb['leukemia'] + x_comb['lymphoma'] + x_comb['solid_tumor_with_metastasis']
        

In [ ]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [ ]:
#Rajesh features

In [ ]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))


In [ ]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


In [ ]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [ ]:
list_cols_nans = list(data_dict[data_dict['Category'].isin(['vitals','labs','labs blood gas'])]['Variable Name'])

In [ ]:
list_cols_nans = list(set(list_cols_nans).intersection(set(x_comb.columns)))
#list_cols_nans

In [ ]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

In [ ]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'labs blood gas']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [ ]:
binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10 ,labels=False))
    binned_cols.append(cols+'_binned')
    

In [ ]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [ ]:
featured_cols = binned_cols + ['age_category','height'+'_binned']

In [ ]:
#2way interaction
bias_cols = ['gender','age']

In [ ]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

In [ ]:
#Critical -3way interaction


x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)
x_comb['age'+'gender'+'ethnicity' +'weighted_classt' ] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str) +'_' + x_comb['weightclass'].astype(str)


inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt'] +['age'+'gender'+'ethnicity'] + ['age'+'gender'+'ethnicity'+'weighted_classt']

In [ ]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']


In [ ]:
inter_cols1 = []

for col in tqdm(list_illness) :
    #x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    #inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

In [ ]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

In [ ]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

In [ ]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

In [ ]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']


In [ ]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

In [ ]:
# aggs = {}
# aggs['apache_4a_hospital_death_prob'] = ['median', 'mean', 'max', 'min']
# aggs['apache_4a_icu_death_prob'] = ['median', 'mean', 'max', 'min']


In [ ]:
new_cat_var = list(set(new_cat_var).intersection(x_comb.columns))

In [ ]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [ ]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [ ]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
def mode(x):
    return Counter(x).most_common(1)[0][0]


In [ ]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = [mode]
    

In [ ]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

In [ ]:
# x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

# le = LabelEncoder()
# x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [ ]:
#x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)
#x_comb = x_comb.drop(list_unimp,axis=1)

In [ ]:
x_comb.shape

In [ ]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [ ]:
# random_state = 42
# np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':6,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':15,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.4, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [ ]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [ ]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    15000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=1500,
                    verbose_eval=200
                    )

In [ ]:
predictions = lgb_clf.predict(test_x)

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/sample_try_24thfeb.csv',index = False)


In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
feature_imp

In [ ]:
list_unimp = list(feature_imp[feature_imp['Value'] == 0]['Feature'])

In [ ]:
feature_imp[feature_imp['Feature'] == 'map_apache']

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=130, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()
# #data_sub

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
oof = np.zeros(len(x_train))
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 13000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val])
    

    
    num_models += 1
    
    oof[valid_index] = clf.predict(train_x.iloc[valid_index])
    predictions += clf.predict(test_x, num_iteration = clf.best_iteration)

In [ ]:
predictions_10 = predictions /10.0
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/final_fold10_dart_24thfeb.csv',index = False)


In [ ]:
#feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
#                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
#feature_imp[feature_imp['Feature'] == 'weightclass']


In [ ]:
#feature_imp[feature_imp['Value'] == 0]

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/sub_10fold_keep_missing_as_it_is_18thFeb2020_newfeat.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_12thfeb.csv')

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4_lgbold_10thfeb_new_bmi_impute.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_10thFeb2020_lgb_type4_ff_20_na_bmi.csv')
# #a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb_type3_ff_27.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_11thFeb2020_lgb_type3_ff_27.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/pred_lgb_keeping_cat_cont_interchanged_11th_feb.csv')
# a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4tillnpw_11thfeb.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death']
           )/2.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_18thfeb.csv',index = False)

In [ ]:
# def hospital_type(x):
    
#     if pd.isna(x):
#         return np.nan
    
#     elif x <= 4:
#         return "less_icu"
    
#     elif x > 8:
#         return "more_icu"
#     else:
#         return "med_icu"
